### Data & Imports

In [6]:
import tensorflow as tf
from typing import List
from sklearn.metrics import confusion_matrix


# data_predicted = [ 0, 0, 0,.5, 1, 1, 1, .5, 0, .5, 0]
# data_truth     = [ 0, 0,.5, 1, 1, 1, .5, 1, .5, 0, 0]

y_pred : List[float] = [[ 0, 0, 0, 1, 1],[ 0, 0, 0, 1, 1]]
y_true : List[float] = [[ 0, 1, 0, 1, 0],[ 0, 1, 0, 1, 0]]


### Using tf.keras.Metrics

In [7]:
m = tf.keras.metrics.Accuracy()
m.update_state(y_true, y_pred)
print(m.result().numpy())

confusion_matrix(y_true, y_pred)

0.6


ValueError: multilabel-indicator is not supported

### Custom Metric as Function

In [4]:
def dice_loss(y_true: List[float], y_pred: List[float]):

  y_true = list(map(float, y_true))
  y_pred = list(map(float, y_pred))

  y_true = tf.cast(y_true, tf.float32)
  y_pred = tf.math.sigmoid(y_pred)
  numerator = 2 * tf.reduce_sum(y_true * y_pred)
  denominator = tf.reduce_sum(y_true + y_pred)

  return 1 - numerator / denominator

dice_loss(y_true,y_pred).numpy()

0.5038172

### Custom Metric as Subclass (recommended)

In [5]:
class DiceLoss(tf.keras.metrics.Metric):

    def __init__(self, name='dice_loss', **kwargs):
        super(DiceLoss, self).__init__(name=name, **kwargs)
        self.dice_loss = None

    def update_state(self, y_true, y_pred, sample_weight=None):

        y_true = list(map(float, y_true))
        y_pred = list(map(float, y_pred))

        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.math.sigmoid(y_pred)
        numerator = 2 * tf.reduce_sum(y_true * y_pred)
        denominator = tf.reduce_sum(y_true + y_pred)

        dice_loss = 1 - numerator / denominator
        self.dice_loss = dice_loss

    def result(self):
        return self.dice_loss

    def reset_states(self):
        self.dice_loss = None

m = DiceLoss()
m.update_state(y_true, y_pred)
m.result().numpy()

0.5038172